In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [23]:
movies_df=pd.read_csv("C:/Users/djroc/OneDrive/Desktop/movies.csv")
ratings_df=pd.read_csv("C:/Users/djroc/OneDrive/Desktop/ratings.csv")
tags_df=pd.read_csv("C:/Users/djroc/OneDrive/Desktop/tags.csv")

In [24]:
movies_df.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [27]:
list=[]
for i in range(len(movies_df['genres'])):
    for genre in movies_df.loc[i,'genres'].split('|'):
        if genre not in list:
            list.append(genre)
    

In [28]:
list

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Mystery',
 'Sci-Fi',
 'War',
 'Musical',
 'Documentary',
 'IMAX',
 'Western',
 'Film-Noir',
 '(no genres listed)']

In [30]:
ratings_df.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [31]:
tags_df.head(5)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [32]:
print(ratings_df.shape)
print(movies_df.shape)
print(tags_df.shape)

(100836, 4)
(9742, 3)
(3683, 4)


In [33]:
combined_movies_ratings=pd.merge(ratings_df,movies_df,on='movieId')
combined_movies_ratings.head(5)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [34]:
combined_movies_ratings.drop('timestamp',inplace=True,axis=1)

In [35]:
combined_movies_ratings.head(5)

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [36]:
combined_movies_ratings.isnull().sum()

userId     0
movieId    0
rating     0
title      0
genres     0
dtype: int64

In [37]:
combined_movies_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
 3   title    100836 non-null  object 
 4   genres   100836 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 4.6+ MB


In [38]:
combined_movie_rating =combined_movies_ratings.dropna(axis = 0, subset = ['title'])
movie_ratingCount=(combined_movie_rating.groupby('title')['rating'].count().reset_index().rename(columns={'rating': 'totalRatingCount'})[['title','totalRatingCount']])

In [39]:
movie_ratingCount.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [40]:
rating_with_totalRatingCount=combined_movies_ratings.merge(movie_ratingCount,on='title',how='inner')

In [41]:
rating_with_totalRatingCount.head(5)

,userId,movieId,rating,title,genres,totalRatingCount
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215


In [42]:
rating_popular_movie=rating_with_totalRatingCount.loc[rating_with_totalRatingCount.totalRatingCount>50,]
rating_popular_movie.shape

(40712, 6)

In [43]:
rating_popular_movie.head(5)

,userId,movieId,rating,title,genres,totalRatingCount
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215


In [44]:
movie_features_df=combined_movies_ratings.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head(5)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
movie_features_df_matrix=csr_matrix(movie_features_df.values)
movie_features_df_matrix

<9719x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100832 stored elements in Compressed Sparse Row format>

In [46]:
knn=NearestNeighbors(metric='cosine')
knn.fit(movie_features_df)

NearestNeighbors(metric='cosine')

In [209]:
#query_index = np.random.choice(movie_features_df.shape[0])
query_index=5713
print(query_index)
distances, indices =knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors =20)

5713


In [210]:
for i in range(0,len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Mission: Impossible - Ghost Protocol (2011):

1: Prometheus (2012), with distance of 0.39337044886866457:
2: Machete (2010), with distance of 0.4230456040081386:
3: Iron Man 3 (2013), with distance of 0.44263725544962196:
4: Mission: Impossible III (2006), with distance of 0.44903996066707996:
5: Cabin in the Woods, The (2012), with distance of 0.457881010333419:
6: Mission: Impossible - Rogue Nation (2015), with distance of 0.4636310748538399:
7: Hurt Locker, The (2008), with distance of 0.4669823281426705:
8: X-Men: Days of Future Past (2014), with distance of 0.48185796427562977:
9: Thor: The Dark World (2013), with distance of 0.48466724887107315:
10: World War Z (2013), with distance of 0.5001930206108489:
11: X-Men: First Class (2011), with distance of 0.5029217322350359:
12: Terminator Genisys (2015), with distance of 0.5038241835862262:
13: Iron Man 2 (2010), with distance of 0.5038645617162171:
14: Skyfall (2012), with distance of 0.5050503398663936:
15: Ki

In [208]:
for i in range(0,movie_features_df.shape[0]):
    if(movie_features_df.index[i]=="Mission: Impossible - Ghost Protocol (2011)"):
        print(i)
        break

5713


In [163]:
movie_features_df.shape

(9719, 610)

# USER BASED-SIMILARITY

In [261]:
movie_features_df=combined_movies_ratings.pivot_table(index='userId',columns='title',values='rating').fillna(0)
movie_features_df.head(5)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [269]:
knn.fit(movie_features_df)

NearestNeighbors(metric='cosine')

In [294]:
query_index=np.random.choice(movie_features_df.shape[0])
print(query_index)
distances, indices=knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1,-1),n_neighbors=10)

99


In [295]:
indices

array([[ 99, 356, 329,  44, 479,  83, 596,  41, 168, 199]], dtype=int64)

In [296]:
distances 

array([[1.11022302e-16, 7.17037623e-01, 7.39625299e-01, 7.43624118e-01,
        7.46041930e-01, 7.48709873e-01, 7.50218677e-01, 7.56619450e-01,
        7.57470813e-01, 7.60058470e-01]])

In [299]:
a=[]
for column in movie_features_df.columns:
    if(movie_features_df.loc[99,column]!=0 and movie_features_df.loc[329,column]!=0):
        a.append(column)

In [300]:
a

['Shawshank Redemption, The (1994)']

In [303]:
columns=["c++","python","html","nodejs","django","kotlin","mongodb","ML","neural networks","CV","java","javascript","javaadvanced","CP","cybersecurity","networking","ethicalhacker","SQL","hadoop","bigdata","androiddeveloper","webdeveloper","kubernet","AWS","Azure"]
df=pd.DataFrame(columns=columns,index=np.arange(1000))

In [305]:
df.fillna(0)

,c++,python,html,nodejs,django,kotlin,mongodb,ML,neural networks,CV,...,networking,ethicalhacker,SQL,hadoop,bigdata,androiddeveloper,webdeveloper,kubernet,AWS,Azure
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [306]:
rand_zero_one_mask = np.random.randint(2, size=df.shape)
# Fill df with 0 where mask is 0
df = df.where(rand_zero_one_mask==0, 0)

In [309]:
df=df.fillna(1)

In [310]:
df.head(5)

,c++,python,html,nodejs,django,kotlin,mongodb,ML,neural networks,CV,...,networking,ethicalhacker,SQL,hadoop,bigdata,androiddeveloper,webdeveloper,kubernet,AWS,Azure
0,1,1,0,0,1,0,1,1,0,0,...,0,1,1,1,0,1,0,1,0,1
1,0,1,1,1,0,1,1,1,0,0,...,1,1,0,1,1,1,0,1,0,0
2,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,1,0,1,1,1,0,0,1,0,...,1,1,1,1,1,1,0,1,0,0
4,1,1,0,0,0,1,0,1,0,0,...,1,0,0,0,1,1,0,0,0,0


In [311]:
KNN=NearestNeighbors(metric='cosine')
KNN.fit(df)

NearestNeighbors(metric='cosine')

In [312]:
query=np.random.choice(df.shape[0])
print(query)
distance, indice=KNN.kneighbors(df.iloc[query,:].values.reshape(1,-1),n_neighbors=10)

752


In [313]:
distance

array([[1.11022302e-16, 1.71921329e-01, 1.71921329e-01, 1.71921329e-01,
        1.94177036e-01, 1.98216274e-01, 1.98216274e-01, 1.98216274e-01,
        1.98216274e-01, 2.14285714e-01]])

In [314]:
indice

array([[752, 227, 281, 380,  92, 944, 147, 536, 820, 579]], dtype=int64)

In [319]:
b=[]
for column in df.columns:
    if(df.loc[752,column]!=0 and df.loc[281,column]!=0):
        b.append(column)

In [320]:
b

['c++',
 'html',
 'nodejs',
 'django',
 'kotlin',
 'CV',
 'javascript',
 'cybersecurity',
 'ethicalhacker',
 'hadoop',
 'webdeveloper',
 'AWS']

In [321]:
c=[]
for column in df.columns:
    if(df.loc[752,column]!=0):
        c.append(column)
print(c)
d=[]
for column in df.columns:
    if( df.loc[281,column]!=0):
        d.append(column)
print(d)

['c++', 'html', 'nodejs', 'django', 'kotlin', 'CV', 'javascript', 'cybersecurity', 'networking', 'ethicalhacker', 'hadoop', 'androiddeveloper', 'webdeveloper', 'AWS']
['c++', 'html', 'nodejs', 'django', 'kotlin', 'CV', 'javascript', 'javaadvanced', 'CP', 'cybersecurity', 'ethicalhacker', 'SQL', 'hadoop', 'webdeveloper', 'AWS']
